In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sunpy
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm 
import sunpy.visualization.colormaps as cm
from astropy.io import fits 
from sunpy.map.header_helper import make_heliographic_header

In [2]:
def standardized_map(file):
    aia_map = sunpy.map.Map(file)
    shape = (1440, 2880)
    carr_header = make_heliographic_header(aia_map.date,
                                           aia_map.observer_coordinate,
                                           shape, frame='stonyhurst',)
    outmap = aia_map.reproject_to(carr_header)
    cc = outmap.data
    flip = cc[::-1,:]
    nan_mask = np.isnan(flip)
    
    center_row = flip.shape[0] // 2

    center_non_nan_cols = np.where(~nan_mask[center_row])[0]
    
    if len(center_non_nan_cols) > 0:
        start_col = center_non_nan_cols[0]
        end_col = center_non_nan_cols[-1] + 1
    
        # Find the first and last non-NaN in the columns of the bounding box
        rows_with_nan_in_box = np.any(~nan_mask[:, start_col:end_col], axis=1)
        start_row = np.argmax(rows_with_nan_in_box)
        end_row = len(rows_with_nan_in_box) - np.argmax(rows_with_nan_in_box[::-1])
    
        cropped_image = flip[start_row:end_row, start_col:end_col]
    else:
        cropped_image = np.array([])  
    
    nan_mask = np.isnan(cropped_image)
    cropped_image[nan_mask] = np.interp(np.flatnonzero(nan_mask), np.flatnonzero(~nan_mask), cropped_image[~nan_mask])

    new_shape = 256
    if cropped_image.shape[0] > 0:
        resized_image = cv2.resize(cropped_image, (new_shape, new_shape), interpolation=cv2.INTER_LINEAR)
    else:
        resized_image = np.full((new_shape, new_shape), np.nan)
    
    # Standardizing the image from the logrithmic scale
    log_i = np.log(resized_image)
    
    # Remove NAN Effects
    non_nan_log_i = log_i[~np.isnan(log_i)]
    stdd = np.std(non_nan_log_i)
    meann = np.mean(non_nan_log_i)
    
    std_log_image = ((log_i - meann)/stdd)
    return std_log_image
    

In [4]:
def process_file_set(file_1, file_2, file_3):
    f_name = file_1[-23:-15]
    std_log_image_171 = standardized_map(file_1)
    std_log_image_193 = standardized_map(file_2)
    std_log_image_304 = standardized_map(file_3)

    #Three Channel 
    three_channel_image = np.stack((std_log_image_171, 
                                    std_log_image_193, 
                                    std_log_image_304), axis=-1)
    
    three_channel_image_float = three_channel_image.astype(np.float32)
           
    output_path = f"C:/Users/Joseph-Judah/Desktop/AIA/Combine_channel/{file_1[-26:-15]}"
   
    
    with fits.HDUList([fits.PrimaryHDU(three_channel_image_float)]) as hdul:
        hdul.writeto(output_path, overwrite=True)
    return f_name
    

def process_directory(year):
    path_1 = f"C:/Users/Joseph-Judah/Desktop/AIA/171/{year}/*.*"
    path_2 = f"C:/Users/Joseph-Judah/Desktop/AIA/193/{year}/*.*"
    path_3 = f"C:/Users/Joseph-Judah/Desktop/AIA/304/{year}/*.*"
   
    
    files_path_1 = sorted(glob.glob(path_1))
    files_path_2 = sorted(glob.glob(path_2))
    files_path_3 = sorted(glob.glob(path_3))
    
    if not (len(files_path_1) == len(files_path_2) == len(files_path_3)):
        raise ValueError(f"The number of files in paths for {year} does not match.")
    
    tasks = []
    with ThreadPoolExecutor() as executor:
        for file_1, file_2, file_3 in zip(files_path_1, files_path_2, files_path_3):
            if file_1[-23:-15] == file_2[-23:-15] == file_3[-23:-15]:
                tasks.append(executor.submit(process_file_set, file_1, file_2, file_3))

        # Display progress with tqdm
        for future in tqdm(as_completed(tasks), total=len(tasks), desc=f"Processing Year={year}"):
            f_name = future.result()  # Get the processed file name
            print(f"Processed: {f_name}")

def main():
    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
    for year in years:
      process_directory(year)




In [ ]:
if __name__ == "__main__":
    main()